In [ ]:
# This notebook performs telescope motion tests in elevation and looks at how the mirror position changes as per the measurements by the PMD
# This is to verify the new M1 LUT (released in v0.8.2.alpha.2)

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS


from lsst.ts.observatory.control.utils import RotType
import logging

In [ ]:
from astropy.time import Time, TimeDelta

In [ ]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
domain = salobj.Domain()

In [ ]:
atcs = ATCS(domain)
# latiss = LATISS(domain)
# await asyncio.gather(atcs.start_task, latiss.start_task)
await asyncio.gather(atcs.start_task)

In [ ]:
await atcs.enable()

In [ ]:
atcs.check.atdome=False
atcs.check.atdometrajectory=False

In [ ]:
tmp=await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.ENABLED, settingsToApply='current', timeout=30)
print(tmp)

In [ ]:
tmp=await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY, settingsToApply='', timeout=30)
print(tmp)

In [ ]:
tmp=await atcs.rem.ataos.evt_settingsApplied.aget()
print(tmp)

### bring the PMDs online

In [ ]:
pmd_remote = salobj.Remote(name="PMD", domain=domain, index=1)
await pmd_remote.start_task

In [ ]:
tmp = await pmd_remote.evt_heartbeat.next(flush=True,timeout=15)
print(tmp)

In [ ]:
tmp = await pmd_remote.cmd_setLogLevel.set_start(level=10)

In [ ]:
tmp = await pmd_remote.evt_summaryState.aget(timeout=15)
print(salobj.State(tmp.summaryState))

In [ ]:
tmp=await salobj.set_summary_state(pmd_remote, salobj.State.ENABLED, settingsToApply='current', timeout=30)
print(tmp)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp.position)

### Setup the script to write log messages
#### note that the index cannot be used anywhere else!

In [ ]:
#start the controller to write script messages and grab the domain
index = os.getuid()*100+np.random.randint(0,10)
print(f'Starting controller using ID = {index}')
c = salobj.Controller(name="Script", index=index)

In [ ]:
c.log.setLevel(logging.DEBUG)

In [ ]:
c.log.info('Patrick Testing INFO level ') # only 

In [ ]:
az=0; el=80
c.log.info(f'az,el,pos0,pos1,pos2,pos3,pos4 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[5]}')

In [ ]:
# Turn on ataos M1 correction, have to turn on hexapod as well due to a bug
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)
await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=False, hexapod=False, atspectrograph=True)

In [ ]:
#verify which is enabled
tmp=await atcs.rem.ataos.evt_correctionEnabled.aget()
print(tmp)

In [ ]:
el_samples2 = np.arange(25,83,7)
el_samples1 = np.arange(83,25,-7)

In [ ]:
# print(el_samples1)
# print(el_samples2)

In [ ]:
el_samples=np.concatenate((el_samples1, el_samples2))

In [ ]:
el_samples

In [ ]:
atcs.check.atdome=False
atcs.check.atdometrajectory=False

In [ ]:
print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[5]}')

In [ ]:
az=0
test_summary='Nominal LUT from m1_hex_20210810_v3.yaml:v0.8.2.alpha.2-0-g72c6e32'
c.log.info(f'Starting {test_summary}')
time=(Time(Time.now(), format='isot', scale='tai'))
print(f'start time is {time})
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[5]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[5]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')
c.log.info(f'End of {test_summary}')
time=(Time(Time.now(), format='isot', scale='tai'))
print(f'Finish time is {time}')

# Now go back up
await atcs.point_azel(az=az, el=el_samples[0])

In [ ]:
await atcs.point_azel(az=az, el=el_samples[0])

In [ ]:
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
print(pressure)
starting_pressure = pressure.pressure

In [ ]:
scale_factor=0.99
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='m1')
await asyncio.sleep(2)
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
c.log.info(f'Pressure reset back to LUT default of {pressure.pressure}')
await atcs.rem.ataos.cmd_offset.set_start(m1=starting_pressure*(1-scale_factor))
await asyncio.sleep(2)
ref_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
test_summary=f'Test2, LUT offset by {starting_pressure*(1-scale_factor)} and is now {ref_pressure.pressure}'

In [ ]:
az=0
c.log.info(f'Starting {test_summary}')
time=(Time(Time.now(), format='isot', scale='tai'))
print(f'start time is {time})
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[5]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[5]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')
c.log.info(f'End of {test_summary}')
time=(Time(Time.now(), format='isot', scale='tai'))
print(f'Finish time is {time}')

# Now go back up
await atcs.point_azel(az=az, el=el_samples[0])

In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='m1')

In [ ]:
atcs.check.atdome=True
atcs.check.atdometrajectory=True

In [ ]:
await atcs.shutdown()